In [1]:
# 목적: 
 # 한 프로세스당 1분 데이터(60만건)를 1시간에 70번을 랜덤한 간격으로 발송
 # 총 30개의 프로세스를 수행할 수 있는 프로그램 작성

# 수행 후 결과 :
 # ES에 데이터 입력
 # 각 프로세스 종료시, 아래의 내용 출력
 # - 
 # - 

In [2]:
# 1. 프로세스 개수 입력받기
# 2. ES에 저장할 document 개수 입력받기
# 3. 프로세스 개수만큼 프로세스 띄우기 
# 4. 1시간에 70번 랜덤한 간격으로 데이터 발송
# 5. ES 데이터 저장
# 6. multi process 수행 
# 7. 각 프로세스 종료 시, 
#  ① 1건 데이터 입력 평균시간, ② 총 데이터 입력 시간, ③ 총 데이터 입력 건수 출력

In [3]:
import numpy as np

import requests
import json

import multiprocessing as mp
import time

In [4]:
def get_sample_noise(noise_file_name):
    
    # 소음데이터 txt 파일 읽기
    # 파일을 읽어 리스트로 만듦
    lines = []
    with open(noise_file_name, "r") as noise_file:
        lines = noise_file.readlines()
    
    # 랜덤한 위치에서 10000 * 60건 읽기   
    start_indices = []
    start_indices.append(np.random.randint(5400000) + 1) # 1은 파일의 앞쪽, 데이터가 아닌 부분을 제거하기 위한 오프셋
    
    # 시작 위치 인덱스 부터 600000개 데이터 추출
    for start_index in start_indices:
        
        bundle = lines[start_index:start_index + 600000]     
        
        bundle_data = []
        for line in bundle:
            # 소음데이터 사이에 개행문자를 제거하기
            stripped = line.rstrip()
            # index열 제거하기
            bundle_data.append(stripped.split(',')[1])
           
    return bundle_data


In [5]:
# 1. 프로세스 개수 입력받기
process_cnt = int(input("프로세스 개수 입력 : "))
print("입력받은 프로세스 개수 : {}".format(process_cnt))

프로세스 개수 입력 : 30
입력받은 프로세스 개수 : 30


In [6]:
# 5. ES 데이터 저장
def save_es(doc_cnt, q):
    
    # 소음데이터 파일
    noise_txt_file = '/src/data/NoiseRawData_10000/0/r_10min.txt'

    noise_list = []
    
    #print("read file: %s" %noise_txt_file)
    sample_noise_list = get_sample_noise(noise_txt_file)
    noise_list = sample_noise_list
    #print("read complete.")
    
    # 'es-raw' 인덱스에 스페이스(' ')를 넣어 하나의 String으로 저장하기
    string_data = ' '.join(noise_list)
    
    headers = {'Content-Type': 'application/json; charset=utf-8'}

    doc = {
            'type': 'text',
            'index': 'false',
            'text': string_data
    }
    
    data = json.dumps(doc)
    
    # data 파일로 저장
    #file = open('/src/hyebin/esnoise/esnoise_json_data(2100).txt', 'w')
    #file.write(data)  
    #file.close()
    
    # 4. 1시간에 70번 랜덤한 간격으로 데이터 발송
    # 1시간 (60 * 60)
    max_time_end = time.time() + (60 * 60)
    
    time_list = []
        
    # 70번
    for i in doc_cnt:
        # 랜덤한 간격
        time.sleep(np.random.randint(0, 50))
        
        print("index : {}".format(i))
        
        # 시작시간
        start_time = time.time()
               
        # https://quickstart-es-http.default:9200
        # http://192.168.10.204:9200
        response = requests.post(url = "https://quickstart-es-http.default:9200/es-raw/_doc/",
                                headers = headers,
                                data = data,
                                verify = False)
        
        # 종료시간
        end_time = time.time()
        
        time_list.append(end_time - start_time)
        #print("start time : {}, end time : {}".format(start_time, end_time))    
            
        if time.time() > max_time_end:
            break

    time_list_np = np.array(time_list)

    q.put(response)
    
    print("time list : {}".format(time_list_np))

    print("--------------------------------")
    print("데이터 1건 입력 평균 시간 : {}".format(np.mean(time_list_np)))


In [7]:
# 2. ES에 저장할 document 개수 입력받기
doc_cnt = int(input("저장할 document 개수 입력 : "))
print("입력받은 document 개수 : {}".format(doc_cnt))
    
# 3. 프로세스 개수만큼 프로세스 띄우기
q_list = [mp.Manager().Queue() for i in range(process_cnt)]

doc_num_list = np.arange(doc_cnt * process_cnt)
print(doc_num_list)
doc_cnt_list = [doc_num_list[i*doc_cnt:(i+1)*doc_cnt] for i in range(process_cnt)]
print(doc_cnt_list)

process_list = [mp.Process(target=save_es, args=(doc_cnt_list[i], q_list[i]), daemon=True) for i in range(process_cnt)]

저장할 document 개수 입력 : 70
입력받은 document 개수 : 70
[   0    1    2 ... 2097 2098 2099]
[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69]), array([ 70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139]), array([140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 

In [8]:
### 6. multi process 수행 

print("Start Processes")
# 시작시간
total_start_time = time.time() 

for process in process_list:
    process.start()

# 프로세스 종료 대기
for process in process_list:
    process.join()

print("***")
print("총 데이터 입력 건수 : {}".format(process_cnt * doc_cnt))
print("총 데이터 입력 시간 : {}".format(time.time() - total_start_time))
print("End Processes")

Start Processes
index : 350


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


index : 0


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


index : 420


...

time list : [24.78270626 14.29063892  8.58698678  3.59843683 23.92686796  6.36778426
  3.37881565  9.73231411 11.99860477  6.88146329  4.69585037  6.02641559
  3.42627835  6.14531088  5.3827548   4.69968772 17.09638882  6.33418655
  7.71356249  7.18324995  7.18125701  4.33796787 13.22088075  4.84593773
  3.01686072  8.01853561  5.47655797  5.90140438  3.58800149  3.14142394
  9.38599086 10.4229517   5.65722919  8.6999476  10.05541015  4.9907577
  5.13485479  2.74518228  6.4015379   6.46474528  7.26753354  4.74583197
  4.86838317  7.38804412 13.96875668 14.99944973  4.57091784  7.62329316
  5.10240865  9.25575018  4.58484459  8.37185526  3.25107932  7.37573242
  5.16774845 10.11862731 10.24650836  9.19156313  8.82315826  4.12173319
 11.0297308   5.39193034  7.11728573 11.44111323 16.69688678  5.75622916
  7.65226078 11.63322568  4.89860058  3.82285428]
--------------------------------
데이터 1건 입력 평균 시간 : 7.820272513798305
index : 279


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


index : 1539


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


time list : [26.52156067  6.36855578  5.61835647  5.29315686  7.31305003 13.64799166
 11.90952969  5.8362987   3.45801878  3.49112988  5.99371338  7.02967191
  8.79637694  7.7761991   9.38768959  5.94915795  6.85323548 16.6671927
  8.1282568   5.64993238  2.95166302  6.70658875  6.14498186 11.94768882
  5.22261834  4.28164339  7.18457866  6.29371428  5.05282354  5.53146052
 13.55226874 17.12905574  6.20062017 13.27142954  5.85220528  2.92432189
  4.64693332  3.53559351  7.85201693  5.69041443  8.32189655  6.31122613
  7.25862741  8.37684369  7.30217957  9.01079988  6.19996166  6.78626347
 10.87425208 16.96483231  6.14438891  5.08572292 10.01683044  5.68584538
  9.10606337  5.21081233  4.04820013 11.07729387  6.22443628 17.34891844
  6.58454466  5.92622352  7.94934964  8.83218098  6.66165876  9.63959002
  7.32078433  7.57826018  5.51948977  2.76160717]
--------------------------------
데이터 1건 입력 평균 시간 : 7.854153990745544
time list : [20.8811667  17.91245294  4.70892644  6.191185   12.708